# Train
Consumes `artifacts/message.json` produced by the upstream `data-preprocess.ipynb` step and logs its contents.


In [ ]:
from pathlib import Path
import json
import logging
import os

# Configure logging
logger = logging.getLogger("train")
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s %(levelname)s [%(name)s] %(message)s"))
    logger.addHandler(handler)
logger.setLevel(os.getenv("LOG_LEVEL", "INFO"))

artifact_path = Path("artifacts/message.json")

if not artifact_path.exists():
    logger.error(f"Artifact not found at {artifact_path}. Ensure Elyra 'File dependencies' includes artifacts/message.json")
    raise SystemExit(2)

with artifact_path.open("r", encoding="utf-8") as f:
    payload = json.load(f)

logger.info(f"Loaded artifact from {artifact_path}")
logger.info(f"Payload keys: {list(payload.keys())}")
logger.info(f"Message: {payload.get('message')}")
logger.info(f"Created at: {payload.get('created_at')}")
logger.info(f"Upstream host: {payload.get('hostname')}")
logger.info(f"Upstream user: {payload.get('user')}")

# Example: use the message to parametrize the run
if payload.get("message"):
    logger.info("Training starts with upstream message acknowledged.")
else:
    logger.warning("No message provided; proceeding with defaults.")


## Elyra node configuration (summary)
- File dependencies: `artifacts/message.json`


In [ ]:
# Optional: demonstrate pip installs if running in a clean image
try:
    import orjson  # lightweight, faster JSON (optional)
    _has_orjson = True
except Exception:
    _has_orjson = False
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "pip", "install", "--no-input", "orjson==3.*"], check=False)
    try:
        import orjson
        _has_orjson = True
    except Exception:
        pass

if _has_orjson:
    logger.info("orjson available; not required, just a speed-up.")
else:
    logger.info("orjson not available; using built-in json.")
